# Index constituent and Ownership

In [1]:
library(DBI)
library(plutoDbR)
library(plutoR)
library(tidyverse)
options("scipen"=999)
options(stringsAsFactors = FALSE)
source("/usr/share/pluto/config.R")
source("/usr/share/pluto/goofy/plot.common.R")
source("/usr/share/pluto/goofy/misc.common.R")

library(pracma)
library(ggthemes)
library(reshape2)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)
library(ggpubr)

options(repr.plot.width=16, repr.plot.height=8)
options(tibble.width = Inf)

equities <- EquitiesIndiaNse()
indices <- Indices()

In [2]:
getShp <- function(sym){
    maxDt <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym) %>%
        summarize(M = max(TIME_STAMP)) %>%
        collect()
    
    maxDt <- as.Date(maxDt$M[1])
    
    PI <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym & TIME_STAMP == maxDt 
               & KEY_2 == "ShareholdingAsAPercentageOfTotalNumberOfShares"
               & (KEY_1 == "ShareholdingOfPromoterAndPromoterGroup"
                  || KEY_1 == "IndividualShareholdersHoldingNominalShareCapitalUpToRsTwoLakh"
                  || KEY_1 == "IndividualShareholdersHoldingNominalShareCapitalInExcessOfRsTwoLakh"
                  || KEY_1 == "IndividualsOrHinduUndividedFamily")) %>%
        summarize(T = sum(VALUE)) %>%
        collect()
    
    MF <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym & TIME_STAMP == maxDt 
               & KEY_2 == "ShareholdingAsAPercentageOfTotalNumberOfShares"
               & KEY_1 == "MutualFundsOrUti") %>%
        summarize(T = sum(VALUE)) %>%
        collect()
    
    return (c(as.numeric(PI$T[1]), as.numeric(MF$T[1])))
}

In [3]:
indexShp <- function(indexName){
    print(indexName)
    constitutents <- indices$NseConstituents() %>%
        filter(NAME == indexName) %>%
        select(SYMBOL, CAP_WEIGHT) %>%
        collect() %>%
        as.data.frame()

    constitutents$PI <- NA
    constitutents$MF <- NA

    for (i in 1:nrow(constitutents)){
        symbol <- toString(constitutents$SYMBOL[i])
        tryCatch({
                pimf <- getShp(symbol)
                constitutents$PI[i] <- pimf[1]
                constitutents$MF[i] <- pimf[2]
            },
            error = function(e){
                print(symbol)
                print(e)
            })
    }

    constitutents$INST <- 100 - (constitutents$PI + constitutents$MF)
    
    constitutents$MF_WT <- constitutents$MF * constitutents$CAP_WEIGHT/100
    constitutents$INST_WT <- constitutents$INST * constitutents$CAP_WEIGHT/100
    
    print("Mutual fund ownership (%)")
    print(summary(constitutents$MF))
    
    print("Institutional ownership (%)")
    print(summary(constitutents$INST))
    
    print("Total MF ownership (%)")
    print(sum(constitutents$MF_WT, na.rm=T))
    
    print("Total Institutional ownership (%)")
    print(sum(constitutents$INST_WT, na.rm=T))
}

In [4]:
indexShp("NIFTY 50")

[1] "NIFTY 50"


[1] "ITC"
<simpleError in as.Date.default(x, ...): do not know how to convert 'x' to class “Date”>
[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.920   5.060   8.080   8.562  10.680  29.660       1 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  14.10   24.68   35.01   37.23   45.65   82.63       1 
[1] "Total MF ownership (%)"
[1] 10.07246
[1] "Total Institutional ownership (%)"
[1] 40.62113


In [5]:
indexShp("NIFTY 100")

[1] "NIFTY 100"


[1] "ITC"
<simpleError in as.Date.default(x, ...): do not know how to convert 'x' to class “Date”>
[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.320   3.365   5.630   6.867   9.410  29.660       1 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 -44.42   19.13   27.41   28.26   36.35   82.63       1 
[1] "Total MF ownership (%)"
[1] 9.472543
[1] "Total Institutional ownership (%)"
[1] 37.74413


In [6]:
indexShp("NIFTY MIDCAP 150")

[1] "NIFTY MIDCAP 150"


[1] "GLENMARK"
<simpleError in as.Date.default(x, ...): do not know how to convert 'x' to class “Date”>
[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.000   3.400   7.090   8.719  12.840  29.670       1 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 -66.69    6.44   16.68   13.20   27.80   58.90       1 
[1] "Total MF ownership (%)"
[1] 11.12162
[1] "Total Institutional ownership (%)"
[1] 18.18821


In [7]:
indexShp("NIFTY SMLCAP 100")

[1] "NIFTY SMLCAP 100"
[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   2.007   5.285   7.946  13.307  36.790 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -61.72    4.79   17.42   15.54   30.20   77.36 
[1] "Total MF ownership (%)"
[1] 9.953712
[1] "Total Institutional ownership (%)"
[1] 19.95408


This notebook was created using [pluto](https://pluto.studio). Thank you for playing fair!